In [1]:
# Import needed libraries and classes

import os

from Code.utils.dataset import Dataset
import Code.utils.store_model as store_model

from tensorflow.keras.datasets import imdb
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import Embedding, Input

# TODO posso usar Tokenizer?
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
# Load datasets

dataset = Dataset('../../Dataset/DatasetsGerados/dataset_training_input.csv',
                  '../../Dataset/DatasetsGerados/dataset_training_output.csv',
                  '../../Dataset/DatasetsGerados/dataset_validation_input.csv',
                  '../../Dataset/DatasetsGerados/dataset_validation_output.csv',
                  '../../Dataset/DatasetsGerados/dataset_test_input.csv',
                  '../../Dataset/DatasetsGerados/dataset_test_output.csv')

X_train, y_train, X_validation, y_validation, X_test, y_test, ids = dataset.get_datasets_embedding('Text', 'Label', sep='\t', rem_punctuation=True)

max_words = 10000
max_len = 500

# print(X_train.iloc[0, 0])
# print(X_train.shape)

tokenizer = Tokenizer(num_words=max_words)  # limit to top max_words words
tokenizer.fit_on_texts(X_train.iloc[:, 0])

X_train = tokenizer.texts_to_sequences(X_train.iloc[:, 0])
X_validation = tokenizer.texts_to_sequences(X_validation.iloc[:, 0])
X_test = tokenizer.texts_to_sequences(X_test.iloc[:, 0])

X_train = preprocessing.sequence.pad_sequences(X_train, maxlen=max_len)
X_validation = preprocessing.sequence.pad_sequences(X_validation, maxlen=max_len)
X_test = preprocessing.sequence.pad_sequences(X_test, maxlen=max_len)

# print(X_train[0])
# print(X_train.shape)

print(y_train)

[1 0 1 ... 0 1 0]


In [3]:
dim_embed = 20
model = Sequential()
model.add(Input((max_len,)))
model.add(Embedding(max_words, dim_embed))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 500, 20)        │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 10000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │        10,001 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 210,001 (820.32 KB)

 Trainable params: 210,001 (820.32 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - acc: 0.8572 - loss: 0.3323 - val_acc: 0.9833 - val_loss: 0.0504
Epoch 2/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - acc: 0.9925 - loss: 0.0290 - val_acc: 0.9875 - val_loss: 0.0391
Epoch 3/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - acc: 0.9978 - loss: 0.0136 - val_acc: 0.9958 - val_loss: 0.0164
Epoch 4/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - acc: 0.9996 - loss: 0.0048 - val_acc: 0.9958 - val_loss: 0.0137
Epoch 5/5
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - acc: 0.9993 - loss: 0.0032 - val_acc: 0.9925 - val_loss: 0.0149


In [4]:
# Evaluate Model

results = model.evaluate(X_test, y_test, verbose = 0)
print(results)

[0.017637111246585846, 0.9944999814033508]


In [5]:
# Predict output

output = model.predict(X_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [6]:
# Store output

results_filepath = './Results/embedding_results.csv'

# Ensure the directory exists
os.makedirs(os.path.dirname(results_filepath), exist_ok=True)

results = dataset.merge_results(ids, output)
results.to_csv(results_filepath, sep='\t', index=False)

In [7]:
# Store network model

model_filepath = './Model/embedding'
model_key = 'embedding'

store_model.store_model(model_filepath, model_key, model)

In [3]:
# Retrieve network model

model_filepath = './Model/embedding'
model_key = 'embedding'

model = store_model.retrieve_model(model_filepath, model_key)